In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml


import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
from openff.toolkit.topology import Molecule
import torch
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import hoomd
import gsd.hoomd
import matplotlib.pyplot as plt

import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/numexpr/expressions.py:21: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _np_version_forbids_neg_powint = LooseVersion(numpy.__version__) >= LooseVersion('1.12.0b1')
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/numexpr/expressions.py:21: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _np_version_forbids_neg_powint = LooseVersion(numpy.__version__) >= LooseVersion('1.12.0b1')
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as so

ModuleNotFoundError: No module named 'gsd'

In [15]:
# Use espaloma to find the FF parameters for Benzene
smiles = "c1ccccc1"
mb_benzene = mb.load(smiles, smiles=True)
#benzene.visualize().show()

#mb_benzene.save("benzene.")
#try to either save molecule as a usable format or get smiles string from mol2 file..

#mb_benzene.save("benzene.sdf")

mb_benzene2 = mb.load("benzene.sdf")

/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/mbuild/conversion.py:800: UserWarning: No unitcell detected for pybel.Molecule [CH]1[CH][CH][CH][CH][CH]1	Compound

  warn(f"No unitcell detected for pybel.Molecule {pybel_mol}")


In [3]:
molecule = Molecule.from_file("benzene.sdf",file_format = "sdf")
#molecule = Molecule.from_smiles(smiles) #molecule is benzene
molecule_graph = esp.Graph(molecule)
#either generate smiles string from the built fragments or get our fragments into a Molecule supported file format

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph)

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]

/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:16: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(th.__version__) < LooseVersion("1.9.0"):
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:16: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(th.__version__) < LooseVersion("1.9.0"):
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:340: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(th.__version__) >= LooseVersion("1.10.0"):
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:340: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(th.__version__) >= Lo

In [4]:
# get a parmed structure from openmm 
import parmed as pmd
topology = molecule.to_topology()
openmm_topology = topology.to_openmm()

structure = pmd.openmm.load_topology(topology=openmm_topology, system=openmm_system)
structure.bonds.sort(key=lambda x: x.atom1.idx)

In [5]:
# step 1: how to get bond parameters

bond_parameters = {}

for bond in structure.bonds:
    bond_parameters[(bond.atom1.name, bond.atom2.name)] = {"k":[bond.type.k] , "l0":[bond.type.req]}
    
# step 2: get angle parameters

angle_parameters = {}

for angle in structure.angles:
    angle_parameters[(angle.atom1.name, angle.atom2.name,angle.atom3.name)] = {"k":[angle.type.k], 
                                                                               "t0":[angle.type.theteq]}
    
# step 3: dihedral parameters

dihedral_parameters = {}

for dihedral in structure.dihedrals:
    dihedral_parameters[(dihedral.atom1.name, dihedral.atom2.name,dihedral.atom3.name,
                         dihedral.atom4.name)] = {"periodicity":[dihedral.type.per],
                                                  "k":[dihedral.type.phi_k],"phase":[dihedral.type.phase]}
    
# step 4: non-bonding parameters

nonbonded_parameters = {}

for nonbonded in structure.adjusts:
    nonbonded_parameters[(nonbonded.atom1.name,nonbonded.atom2.name)] = {"sigma":[nonbonded.type.rmin],
                                                    "epsilon":[nonbonded.type.epsilon],
                                                    "charge":[nonbonded.type.chgscale]}

In [6]:
particle_types = []
particle_type_dict = dict()

for i in range(pair_forces.getNumParticles()):
    pair_parms = pair_forces.getParticleParameters(index=i)
    charge = pair_parms[0]
    sigma = pair_parms[1]
    epsilon = pair_parms[2]
    if (charge, sigma, epsilon) not in particle_types: #this is adding the charge,sigma, and epsilon to the particle_types, but only once.
        particle_types.append((charge, sigma, epsilon))
        #print((charge, sigma, epsilon) )
    particle_type_dict[i] = particle_types.index((charge, sigma, epsilon)) #adding only the index of the pair_parms, this gives particle types.
print(particle_type_dict)

#particle_types gives the sigma, charge, and epsilon associated with each unique particle, 
#particle_type_dict only gives the index associated with each unique particle type

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1}


In [7]:
#graph check
#want to check if the opemm and parmed graphs are the same so we can use types from parmed 
import networkx  as nx
Gopenmm = nx.Graph()
Gparmed = nx.Graph()
#openmm:
for i in range(bond_forces.getNumBonds()):
    Gopenmm.add_edge(bond_forces.getBondParameters(index=i)[0],bond_forces.getBondParameters(index=i)[1])
#parmed
for b in structure.bonds:
    Gparmed.add_edge(b.atom1.idx,b.atom2.idx)
type_map = {}
#nx.rooted_tree_isomorphism
#in here we still need to check that one known index on one corresponds to the same index on the other....
tree_openmm = nx.bfs_tree(Gopenmm,0)
tree_parmed = nx.bfs_tree(Gparmed,0)
if nx.is_isomorphic(Gopenmm,Gparmed):
#if nx.isomorphism.tree_isomorphism(tree_openmm,tree_parmed):  <- want this work
    for b in structure.bonds:
        type_map[b.atom1.idx] = b.atom1.atom_type.__str__()
        type_map[b.atom2.idx] = b.atom2.atom_type.__str__()
print(type_map)

{1: 'C1', 0: 'C1', 2: 'C1', 3: 'C1', 4: 'C1', 5: 'C1', 6: 'H1', 7: 'H1', 8: 'H1', 9: 'H1', 10: 'H1', 11: 'H1'}


In [8]:
# Rename the particle types so that they match the xml file
# This is needed when we aren't using SMARTS matching with Foyer.

for index in type_map:
    mb_benzene[index].name = type_map[index]

In [9]:
bond_types = []
bond_dict = dict() #creating an empty dictionary to put our parameters in 

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    #print(bond_parms)
    #atoms = {(bond.atom1.name,bond.atom2.name):[]} #gives you the atom type name and index
    l0 = bond_parms[2]/bond_parms[2].unit
    k = bond_parms[3]/bond_parms[3].unit
    #if (type_map[bond_parms[0]],type_map[bond_parms[1]],k,l0) not in bond_dict:
    #    bond_types.append((type_map[bond_parms[0]],type_map[bond_parms[1]],k,l0))
    bond_dict[type_map[bond_parms[0]],type_map[bond_parms[1]]] = {'k':k,'l0':l0}
print(bond_dict)

{('C1', 'C1'): {'k': 145133.81359445286, 'l0': 0.14950230459380798}, ('C1', 'H1'): {'k': 297486.872594642, 'l0': 0.10608345867182073}}


In [10]:
angle_types = []
angle_dict = dict() #creating an empty dictionary to put our parameters in 

for i in range(angle_forces.getNumAngles()):
    angle_parms = angle_forces.getAngleParameters(index=i)
    k = angle_parms[4]/angle_parms[4].unit
    t0 = angle_parms[3]/angle_parms[3].unit  
#    if (type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]],k,t0) not in angle_types:
#        angle_types.append((type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]],k,t0))
    angle_dict[type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]]] = {'k':k,'t0':t0}
print(angle_dict)

{('C1', 'C1', 'C1'): {'k': 351.76117106534105, 't0': 1.918205738067627}, ('C1', 'C1', 'H1'): {'k': 360.1267624857336, 't0': 1.9589132070541382}}


In [11]:
dihedral_types = []
dihedral_dict = {}

for i in range(torsion_forces.getNumTorsions()):
    if i%6==0:
        periodicity=[]
        phase = []
        k = []
    dihedral_parms = torsion_forces.getTorsionParameters(index=i)
    periodicity.append(dihedral_parms[4])  
    phase.append( dihedral_parms[5]/dihedral_parms[5].unit)
    k.append(dihedral_parms[6]/dihedral_parms[6].unit)
#    if (type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],type_map[dihedral_parms[3]],periodicity,k,phase) not in dihedral_types:
#        dihedral_types.append((type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
#        type_map[dihedral_parms[3]],periodicity,k,phase))
    dt = (type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
                  type_map[dihedral_parms[3]])
   

    if periodicity[-1]==6:
        #print(dt,periodicity,phase)
        dihedral_dict[dt] = {'periodicity':periodicity,'k':k,'phase':phase}
print(dihedral_dict)

{('C1', 'C1', 'C1', 'C1'): {'periodicity': [1, 2, 3, 4, 5, 6], 'k': [0.44826881252873496, 1.5596955914863755, 1.5069883612995336, 1.2375580868996021, 0.011805363241032322, 0.05734697267852756], 'phase': [0.0, 3.141592653589793, 0.0, 0.0, 0.0, 3.141592653589793]}, ('C1', 'C1', 'C1', 'H1'): {'periodicity': [1, 2, 3, 4, 5, 6], 'k': [1.6213986649548573, 1.2316847475240926, 1.5717130747069354, 0.8619186663237219, 2.2819463941717077, 0.5420344016347554], 'phase': [3.141592653589793, 3.141592653589793, 3.141592653589793, 0.0, 0.0, 3.141592653589793]}, ('H1', 'C1', 'C1', 'H1'): {'periodicity': [1, 2, 3, 4, 5, 6], 'k': [2.428503572418964, 2.3115466064157664, 0.22902600302569417, 0.10181759017327793, 0.4002771256331796, 0.21289999420742403], 'phase': [0.0, 3.141592653589793, 0.0, 0.0, 3.141592653589793, 3.141592653589793]}, ('C1', 'C1', 'H1', 'C1'): {'periodicity': [1, 2, 3, 4, 5, 6], 'k': [1.6213913293936055, 1.2316847475240926, 1.571720410268187, 0.8619235566978897, 2.2819268326750364, 0.54203

In [12]:
nonbonded_types = []
nonbonded_dict = {}

for i in range(pair_forces.getNumParticles()):
    nonbonded_parms = pair_forces.getParticleParameters(index=i)
    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
    epsilon = nonbonded_parms[2]/nonbonded_parms[2].unit
    #atom_type = structure.adjusts.atom
    if (charge,sigma,epsilon) not in nonbonded_types:
        nonbonded_types.append((charge,sigma,epsilon))
    nonbonded_dict[(type_map[i])]={'charge':charge,'sigma':sigma,'epsilon':epsilon}
print(nonbonded_dict)

{'C1': {'charge': -0.136667, 'sigma': 0.3399669508423535, 'epsilon': 0.359824}, 'H1': {'charge': 0.136667, 'sigma': 0.25996424595335105, 'epsilon': 0.06276}}


In [16]:
# Save the forcefield XML file for future use, so that we don't have to repeat the espaloma process everytime
mbuild_to_foyer_xml(
    file_name="benzene_esp2.xml",
    compound=mb_benzene2,
    bond_params=bond_dict,
    angle_params=angle_dict,
    dihedral_params=dihedral_dict,
    dihedral_type="periodic",
    non_bonded_params=nonbonded_dict,
    combining_rule="geometric",
    name="",
    version="",
    coulomb14scale=1.0,
    lj14scale=1.0)

# Save the mb.Compound with the new atom type names for future use.
mb_benzene.save("benzene_typed.mol2", overwrite=True)

/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1225  0.0663 -0.0129]), 3 bonds, id: 140582873023440> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1195 -0.0727  0.0023]), 3 bonds, id: 140582873153104> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([ 0.003  -0.1391  0.0151]), 3 bonds, id: 140590467114240> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/madilyn/miniconda3/envs/espaloma/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([ 0.1225 -0.0663  0.0129]), 3 bonds, id: 1405

## Workflow once we have created an XML file, and a typed mol2 file:

Everything above this cell is something that we should only need to do once.
Then, once we have an XML file, and a typed mol2 file (i.e. one with the particles renamed correctly)
every time we want to run a new simulation, the process would look like the cells below:

In [ ]:
import mbuild as mb
import foyer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

In [ ]:
esp = foyer.Forcefield(forcefield_files="benzene_esp.xml")

In [ ]:
# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
benzene = mb.load("benzene_typed.mol2")
for p in benzene.particles():
    p.name = f"_{p.name}"

In [ ]:
box = mb.fill_box(compound=benzene, n_compounds=5, box=[1,1,1])
box_pmd = esp.apply(box)

In [ ]:
snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)

In [ ]:
#setting cpu and simulation 
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

In [ ]:
#setting the integrator
kt = 4
free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt=0.01)
nvt = hoomd.md.methods.NVT(kT=kt,filter=free_particle, tau=0.1)
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=free_particle, kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_benzene.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)